# BoneawareAI

Authors: Karthik Subramanian, Charles Green, Sai Anurag Pichika, Saarang Prabhuram


## Setup

### Load Extensions

Before getting started we need to run some standard code to set up our environment. You'll need to execute this code again each time you start the notebook.

First, run this cell to load the [autoreload](https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html?highlight=autoreload) extension. This enables us to modify `.py` source files and reintegrate them into the notebook, ensuring a smooth editing and debugging experience.

In [29]:
%load_ext autoreload
%autoreload 2

### Google Colab Setup
Next we need to run a few commands to set up our environment on Google Colab. If you are running this notebook on a local machine you can skip this section.

Run the following cell to mount your Google Drive. Follow the link, sign in to your Google account (the same account you used to store this notebook!).

In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
import os
PROJECT_PATH = 'BoneawareAI'
GOOGLE_DRIVE_PATH = f'/content/drive/MyDrive/{PROJECT_PATH}'
os.chdir(GOOGLE_DRIVE_PATH)
os.getcwd()

'/content/drive/MyDrive/BoneawareAI'

In [32]:
import sys
sys.path.append(GOOGLE_DRIVE_PATH) # this is important for the imports in the .py files to work

In [33]:
%pip install pyyaml==5.4.1
%pip install boto3
%pip install configparser
%pip install torch

  Using cached PyYAML-5.4.1.tar.gz (175 kB)
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Getting requirements to build wheel ... error
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


### Local Setup OR Google Drive
Run the cell below regardless of whether you are using google drive or local setup.

In [34]:
# if running locally set GOOGLE PATH
import sys
isLocal = False
if 'google.colab' in sys.modules:
  print(f'Running in google colab. Our path is `{GOOGLE_DRIVE_PATH}`')
else:
  GOOGLE_DRIVE_PATH = '.'
  print('Running locally.')
  isLocal = True

Running in google colab. Our path is `/content/drive/MyDrive/BoneawareAI`


### Imports

In [35]:
# RUN LOCALLY
import sys
if isLocal:
    sys.path.append('../src')  # Add the 'src' folder to Python's module search path
    sys.path.append('../datasets')  # Add the 'datasets' folder to Python's module search path
    sys.path.append('../notebooks')  # Add the 'notebooks' folder to Python's module search path
    print('Modules added correctly, locally.')
else:
    sys.path.append('src')  # Add the 'src' folder to Python's module search path
    sys.path.append('datasets')  # Add the 'datasets' folder to Python's module search path
    sys.path.append('notebooks')  # Add the 'notebooks' folder to Python's module search path
    print('Modules added correctly on colab.')

Modules added correctly on colab.


In [36]:
from image_utils import set_seed, MURADataset, load_data, confirm_images_and_labels, count_body_parts, count_positive_negative, count_body_parts_with_augmentations

In [37]:
import torch

device = 'mps' if torch.backends.mps.is_available() else ('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device = " + device)
if device == 'cpu':
    print("WARNING: Using CPU will cause slower train times")


Using device = cuda


#### Set Seed

This is so the results can be duplicated, ensure that the seed is set in the image_utils.py file, if you want a random seed, import random and set a random number

In [38]:
set_seed(42)

In [39]:
import os
import torch
import random
import numpy as np
import pandas as pd
from torch import nn, optim

## Data Preprocessing
Get the dataset, perform data augmentation to get finalized MURA dataset

In [45]:
# Downloading MURA dataset and unzipping the file (this one takes time)
from data_loader import download_dataset
from constants import DATASETS_FOLDER, MURA_DATASET
from helpers.utils import unzip_file
# Define the parent directory and dataset path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))  # Go to the parent directory
datasets_folder = os.path.join(parent_dir, DATASETS_FOLDER)   # Define datasets folder in the parent directory
dataset_path = os.path.join(datasets_folder, MURA_DATASET)    # Full path to the dataset file

# Ensure the datasets folder exists
os.makedirs(datasets_folder, exist_ok=True)

# Check if the dataset is already downloaded
if not os.path.exists(dataset_path):
    print(f"{MURA_DATASET} not found in {DATASETS_FOLDER}. Downloading and extracting...")
    # Download and unzip the dataset
    download_dataset(MURA_DATASET, datasets_folder)
    unzip_file(dataset_path)
else:
    print(f"{MURA_DATASET} already exists in {DATASETS_FOLDER}. Skipping download.")

MURA-v1.1.zip not found in datasets. Downloading and extracting...
File downloaded successfully to /content/drive/MyDrive/datasets/MURA-v1.1.zip
successfully unzipped the file at path /content/drive/MyDrive/datasets/MURA-v1.1.zip


In [ ]:

data_dir = "../datasets/MURA-v1.1"
batch_size = 32

# Load training and validation data
train_loader, valid_loader = load_data(data_dir, batch_size=batch_size)

Loaded 147232 training samples and 3197 validation samples.


In [ ]:
print("Training Data:")
for images, labels in train_loader:
    print(f"Batch size: {len(images)}, Labels: {labels}")
    break

# Test the validation DataLoader
print("Validation Data:")
for images, labels in valid_loader:
    print(f"Batch size: {len(images)}, Labels: {labels}")
    break

Training Data:
Batch size: 32, Labels: tensor([0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 1, 0])
Validation Data:
Batch size: 32, Labels: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])


In [ ]:
# Access the datasets from the DataLoaders
train_dataset = train_loader.dataset
valid_dataset = valid_loader.dataset

# Example: Print the length of the datasets
print(f"Number of samples in the training dataset: {len(train_dataset)}")
print(f"Number of samples in the validation dataset: {len(valid_dataset)}")

Number of samples in the training dataset: 147232
Number of samples in the validation dataset: 3197


In [ ]:
#16 minutes to confirm on local, does not need to run as you can always use the dataset to confirm as well
#confirm_images_and_labels(train_dataset, "train")
#confirm_images_and_labels(valid_dataset, "valid")

In [ ]:
count_body_parts(train_dataset, "train")
count_body_parts(valid_dataset, "valid")

Train dataset body part distribution:


,BodyPart,Count
0,XR_WRIST,9752
1,XR_SHOULDER,8379
2,XR_HAND,5543
3,XR_FINGER,5106
4,XR_ELBOW,4931
5,XR_FOREARM,1825
6,XR_HUMERUS,1272


Valid dataset body part distribution:


,BodyPart,Count
0,XR_WRIST,659
1,XR_SHOULDER,563
2,XR_ELBOW,465
3,XR_FINGER,461
4,XR_HAND,460
5,XR_FOREARM,301
6,XR_HUMERUS,288


In [ ]:
# Example usage with 3 augmentations,  adjust the augmentations as needed
count_body_parts_with_augmentations(train_dataset, "train", num_augmentations=3)
count_body_parts_with_augmentations(valid_dataset, "valid", num_augmentations=3)

Train dataset body part distribution (with augmentations):


,BodyPart,OriginalCount,AugmentedCount
0,XR_WRIST,9752,39008
1,XR_SHOULDER,8379,33516
2,XR_HAND,5543,22172
3,XR_FINGER,5106,20424
4,XR_ELBOW,4931,19724
5,XR_FOREARM,1825,7300
6,XR_HUMERUS,1272,5088


Valid dataset body part distribution (with augmentations):


,BodyPart,OriginalCount,AugmentedCount
0,XR_WRIST,659,2636
1,XR_SHOULDER,563,2252
2,XR_ELBOW,465,1860
3,XR_FINGER,461,1844
4,XR_HAND,460,1840
5,XR_FOREARM,301,1204
6,XR_HUMERUS,288,1152


In [ ]:
# Count positive/negative cases in the training dataset (with 3 augmentations)
count_positive_negative(train_dataset, "train", num_augmentations=3)

# Count positive/negative cases in the validation dataset (with 3 augmentations)
count_positive_negative(valid_dataset, "valid", num_augmentations=3)

Train dataset positive/negative distribution (with augmentations):


,BodyPart,Negative,Positive,AugmentedNegative,AugmentedPositive
0,XR_ELBOW,2925,2006,11700,8024
1,XR_FINGER,3138,1968,12552,7872
2,XR_FOREARM,1164,661,4656,2644
3,XR_HAND,4059,1484,16236,5936
4,XR_HUMERUS,673,599,2692,2396
5,XR_SHOULDER,4211,4168,16844,16672
6,XR_WRIST,5765,3987,23060,15948


Valid dataset positive/negative distribution (with augmentations):


,BodyPart,Negative,Positive,AugmentedNegative,AugmentedPositive
0,XR_ELBOW,235,230,940,920
1,XR_FINGER,214,247,856,988
2,XR_FOREARM,150,151,600,604
3,XR_HAND,271,189,1084,756
4,XR_HUMERUS,148,140,592,560
5,XR_SHOULDER,285,278,1140,1112
6,XR_WRIST,364,295,1456,1180


### Other Datasets

: 

# Model

###DenseNet

In [ ]:
# 5. Define Custom DenseNet
class DenseLayer(nn.Module):
    def __init__(self, in_channels, growth_rate, dropout_rate=0.2):
        super(DenseLayer, self).__init__()
        self.layer = nn.Sequential(
            nn.BatchNorm2d(in_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels, growth_rate, kernel_size=3, stride=1, padding=1, bias=False),
            nn.Dropout(dropout_rate)
        )

    def forward(self, x):
        new_features = self.layer(x)
        return torch.cat([x, new_features], dim=1)

class DenseBlock(nn.Module):
    def __init__(self, num_layers, in_channels, growth_rate, dropout_rate=0.2):
        super(DenseBlock, self).__init__()
        layers = []
        for _ in range(num_layers):
            layers.append(DenseLayer(in_channels, growth_rate, dropout_rate))
            in_channels += growth_rate
        self.block = nn.Sequential(*layers)

    def forward(self, x):
        return self.block(x)

class TransitionLayer(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(TransitionLayer, self).__init__()
        self.transition = nn.Sequential(
            nn.BatchNorm2d(in_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False),
            nn.AvgPool2d(kernel_size=2, stride=2)
        )

    def forward(self, x):
        return self.transition(x)

class DenseNet(nn.Module):
    def __init__(self, num_blocks, num_layers_per_block, growth_rate, reduction, num_classes=1):
        super(DenseNet, self).__init__()
        self.growth_rate = growth_rate
        in_channels = 2 * growth_rate

        # Initial Convolution
        self.init_conv = nn.Sequential(
            nn.Conv2d(3, in_channels, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(in_channels),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )

        # DenseBlocks with Transition Layers
        blocks = []
        for i in range(num_blocks):
            blocks.append(DenseBlock(num_layers_per_block, in_channels, growth_rate))
            in_channels += num_layers_per_block * growth_rate
            if i != num_blocks - 1:  # No transition after the last block
                out_channels = int(in_channels * reduction)
                blocks.append(TransitionLayer(in_channels, out_channels))
                in_channels = out_channels

        self.features = nn.Sequential(*blocks)

        # Classification Layer
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(in_channels, num_classes),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.init_conv(x)
        x = self.features(x)
        x = self.classifier(x)
        return x

In [ ]:
# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Instantiate the DenseNet
model = DenseNet(num_blocks=3, num_layers_per_block=4, growth_rate=32, reduction=0.5).to(device)

In [ ]:
# 6. Define Loss, Optimizer, and Scheduler
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# 7. Training Function with Progress Monitoring
def train_model(model, criterion, optimizer, train_loader, valid_loader, num_epochs=25):
    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        print('-' * 10)

        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()
                loader = train_loader
            else:
                model.eval()
                loader = valid_loader

            running_loss = 0.0
            running_corrects = 0

            progress_bar = tqdm(enumerate(loader), total=len(loader), desc=f"{phase} Progress")

            for i, (inputs, labels) in progress_bar:
                # Skip batches with problematic files
                inputs = [inp for inp in inputs if inp is not None]
                labels = [lbl for lbl in labels if lbl is not None]

                if len(inputs) == 0 or len(labels) == 0:
                    continue

                inputs = torch.stack(inputs).to(device)
                labels = torch.tensor(labels).to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    outputs = outputs.squeeze()
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                preds = (outputs > 0.5).float()
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels)

                progress_bar.set_postfix(loss=loss.item())

            epoch_loss = running_loss / len(loader.dataset)
            epoch_acc = running_corrects.double() / len(loader.dataset)

            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

    print(f"Best Validation Accuracy: {best_acc:.4f}")
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
# 8. Train the Model
model = train_model(model, criterion, optimizer, train_loader, valid_loader, num_epochs=10)

In [ ]:
# Save the model
torch.save(model.state_dict(), 'densenet_mura.pth')

# Load the model
model.load_state_dict(torch.load('densenet_mura.pth'))
model.eval()

In [ ]:
from sklearn.metrics import classification_report, roc_auc_score

def evaluate_model(model, loader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.float().to(device)
            outputs = model(inputs).squeeze()
            preds = (outputs > 0.5).float()

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    print("Classification Report:")
    print(classification_report(all_labels, all_preds))
    print(f"AUC-ROC: {roc_auc_score(all_labels, all_preds):.4f}")

# Evaluate on validation set
evaluate_model(model, valid_loader)